<a href="https://colab.research.google.com/github/lowfuel/DiscoDiffusion-Warp-gobig/blob/lowfuel/CLIP_Evaluator.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# CLIP Evaluator

Please considering supporting me [Patreon](https://www.patreon.com/user?u=255893&fan_landing=true) to keep this notebook updated and improving. Thanks!

This notebook allows you to provide some sample pieces of art, then have CLIP evaluate the images and tell you who it thinks the artist is.

Please be sure to add the actual artist to the list if they aren't already there, and message me on Discord (lowfuel) if you have an artist I should add to the list.

## How to interpret the results

- If CLIP agrees that your artist created the images, you can be confident in using that artist's name in your prompts.
- If CLIP disagrees, it means that some other artist (in our list) is generating a response that is more similar to your source art.
-- This doesn't mean your artist is not well understood by CLIP, but it may mean that another artist (or artists) have a better representation in the models compared to yours.
- If CLIP disagrees, and the artist it thinks made it is also a poor match, it could mean that there was no stong artistic recognition, but perhaps some other element of the image triggered a response.

### Credits & Changelog ⬇️

#### Credits

by Jason Hough (lowfuel)

#### License

Licensed under the MIT License

Copyright (c) 2022 Jason Hough 

Permission is hereby granted, free of charge, to any person obtaining a copy
of this software and associated documentation files (the "Software"), to deal
in the Software without restriction, including without limitation the rights
to use, copy, modify, merge, publish, distribute, sublicense, and/or sell
copies of the Software, and to permit persons to whom the Software is
furnished to do so, subject to the following conditions:

The above copyright notice and this permission notice shall be included in
all copies or substantial portions of the Software.

THE SOFTWARE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND, EXPRESS OR
IMPLIED, INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF MERCHANTABILITY,
FITNESS FOR A PARTICULAR PURPOSE AND NONINFRINGEMENT. IN NO EVENT SHALL THE
AUTHORS OR COPYRIGHT HOLDERS BE LIABLE FOR ANY CLAIM, DAMAGES OR OTHER
LIABILITY, WHETHER IN AN ACTION OF CONTRACT, TORT OR OTHERWISE, ARISING FROM,
OUT OF OR IN CONNECTION WITH THE SOFTWARE OR THE USE OR OTHER DEALINGS IN
THE SOFTWARE.

# 1. Set Up

In [ ]:
#@title 1.1 Check GPU Status
#@markdown Note: Works fine without GPU (just a bit slow)
import subprocess
simple_nvidia_smi_display = True#@param {type:"boolean"}
if simple_nvidia_smi_display:
  #!nvidia-smi
  nvidiasmi_output = subprocess.run(['nvidia-smi', '-L'], stdout=subprocess.PIPE).stdout.decode('utf-8')
  print(nvidiasmi_output)
else:
  #!nvidia-smi -i 0 -e 0
  nvidiasmi_output = subprocess.run(['nvidia-smi'], stdout=subprocess.PIPE).stdout.decode('utf-8')
  print(nvidiasmi_output)
  nvidiasmi_ecc_note = subprocess.run(['nvidia-smi', '-i', '0', '-e', '0'], stdout=subprocess.PIPE).stdout.decode('utf-8')
  print(nvidiasmi_ecc_note)

In [ ]:
#@title 1.2 Prepare Folders
import subprocess, os, sys, ipykernel

def gitclone(url):
  res = subprocess.run(['git', 'clone', url], stdout=subprocess.PIPE).stdout.decode('utf-8')
  print(res)

def pipi(modulestr):
  res = subprocess.run(['pip', 'install', modulestr], stdout=subprocess.PIPE).stdout.decode('utf-8')
  print(res)

def pipie(modulestr):
  res = subprocess.run(['git', 'install', '-e', modulestr], stdout=subprocess.PIPE).stdout.decode('utf-8')
  print(res)

def wget(url, outputdir):
  res = subprocess.run(['wget', url, '-P', f'{outputdir}'], stdout=subprocess.PIPE).stdout.decode('utf-8')
  print(res)

import os

try:
    from google.colab import drive
    print("Google Colab detected. Using Google Drive.")
    is_colab = True
    #@markdown If you connect your Google Drive, you can save the final image of each run on your drive.
    google_drive = True #@param {type:"boolean"}
    #@markdown Click here if you'd like to save the diffusion model checkpoint file to (and/or load from) your Google Drive:
    save_models_to_google_drive = True #@param {type:"boolean"}
except:
    is_colab = False
    google_drive = False
    save_models_to_google_drive = False
    print("Google Colab not detected.")

if is_colab:
    if google_drive is True:
        drive.mount('/content/drive')
        root_path = '/content/drive/MyDrive/AI/Disco_Diffusion'
    else:
        root_path = '/content'
else:
    root_path = os.getcwd()

import os
def createPath(filepath):
    os.makedirs(filepath, exist_ok=True)

initDirPath = f'{root_path}/init_images'
createPath(initDirPath)
outDirPath = f'{root_path}/images_out'
createPath(outDirPath)

if is_colab:
    if google_drive and not save_models_to_google_drive or not google_drive:
        model_path = '/content/models'
        createPath(model_path)
    if google_drive and save_models_to_google_drive:
        model_path = f'{root_path}/models'
        createPath(model_path)
else:
    model_path = f'{root_path}/models'
    createPath(model_path)

if os.path.exists(f"{root_path}/prompts.txt"):
  os.remove(f"{root_path}/prompts.txt")

wget("https://raw.githubusercontent.com/lowfuel/DiscoDiffusion-Warp-gobig/lowfuel/prompts.txt", root_path)

def createPath(filepath):
    os.makedirs(filepath, exist_ok=True)


In [ ]:
#@title ### 1.3 Install and import dependencies

import pathlib, shutil, sys

PROJECT_DIR = os.path.abspath(os.getcwd())

multipip_res = subprocess.run(['pip', 'install', 'lpips', 'datetime', 'timm', 'ftfy', 'einops', 'pytorch-lightning', 'omegaconf' ], stdout=subprocess.PIPE).stdout.decode('utf-8')
print(multipip_res)

import os
from os import path
import sys

from google.colab import files

from attr import has
from numpy import average
import torch
from torch import nn
import ipywidgets as widgets
from IPython import display
import requests
try:
  from CLIP import clip
except:
  if not os.path.exists("CLIP"):
    gitclone("https://github.com/openai/CLIP")
  sys.path.append(f'{PROJECT_DIR}/CLIP')
  from CLIP import clip
import gc
from statistics import mean
from PIL import Image, ImageOps
import urllib.request, urllib.error, urllib.parse
from itertools import chain, islice

DEVICE = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print('Using device:', DEVICE)
device = DEVICE # At least one of the modules expects this name..

if 'cuda' in str(device):
  if torch.cuda.get_device_capability(DEVICE) == (8,0): ## A100 fix thanks to Emad
    print('Disabling CUDNN for A100 gpu', file=sys.stderr)
    torch.backends.cudnn.enabled = False


# 2. Select your CLIP models

Selecting them all is fine, but might take a while.

In [ ]:
ViTB32 = True #@param{type:"boolean"}
ViTB16 = True #@param{type:"boolean"}
ViTL14 = True #@param{type:"boolean"}
ViTL14_336 = True #@param{type:"boolean"}
RN101 = True #@param{type:"boolean"}
RN50 = True #@param{type:"boolean"}
RN50x4 = True #@param{type:"boolean"}
RN50x16 = True #@param{type:"boolean"}
RN50x64 = True #@param{type:"boolean"}

# 3. Enter prompt and select sample images

In [ ]:
#@markdown Which artist would you like to test? Enter their name first, then "run" this section and upload a sample
#@markdown Note: If you don't know their name and just want to see who CLIP matches to this sample, enter a fake name
evalprompt = "Peter Mohrbacher" #@param{type:"string"}

#@markdown Upload a sample after the button appears:
uploaded = files.upload()

# 4. RESULTS

In [ ]:
modellist = []
if RN50x64 == True: modellist.append('RN50x64')
if ViTB16 == True: modellist.append('ViT-B/16')
if ViTL14 == True: modellist.append('ViT-L/14')
if ViTL14_336 == True: modellist.append('ViT-L/14@336px')
if RN50 == True: modellist.append('RN50')
if RN50x4 == True: modellist.append('RN50x4')
if RN50x16 == True: modellist.append('RN50x16')
if ViTB32 == True: modellist.append('ViT-B/32')
if RN101 == True: modellist.append('RN101')

def load_clip_model(model_name):
    model, preprocess = clip.load(model_name, jit=False, download_root=model_path,device=device)
    return model, preprocess

def clipit_text(prompt, model):
    text = model.encode_text(clip.tokenize(prompt).to(device)).float()
    return text

def clipit_image(im_prompt, model):
    image = model.encode_image(im_prompt.to(device)).float()
    return image

def evalprompt_cos(t_prompt, t_comp):
    #Get similarity between two tensors
    cos = nn.CosineSimilarity(dim=1, eps=1e-6)
    t_con = cos(t_prompt, t_comp)
    similarity = t_con.item()
    return similarity

def match_prompts_to_image(image, prompts, model):
    t_prompts = clip.tokenize(prompts).to(device)
    with torch.no_grad():
        prompt_features = model.encode_text(t_prompts)
        image_features = model.encode_image(image)
    with torch.no_grad():
        logits_per_image, logits_per_text = model(image, t_prompts)
        probs = logits_per_image.softmax(dim=-1).cpu().numpy()
    return probs

def Average(lst):
    return sum(lst) / len(lst)

def loadprompts(textfile):
    prompts = []
    with open(textfile, encoding="utf-8") as f:
        for line in f:
            prompts.append(line.strip())
    return(prompts)
    
def scoreprompts(evalprompt, images, prompts, modelname):
    model, preprocess = load_clip_model(modelname)
    scores = []
    #print(images[0])
    test_image = preprocess(Image.open(images[0])).unsqueeze(0).to(device)
    probabilities = match_prompts_to_image(test_image, prompts, model)
    problist = (probabilities.tolist())[0]
    for count, prompt in enumerate(prompts):
        prob = round(problist[count], 4)
        if prob > 0.03 or prompt == evalprompt:
            scores.append((prompt, prob))
            #print(f'Probability: {prompt} - {prob}')
    return scores

images = []
for fn in uploaded.keys():
    images.append(fn)

prescript = "by"
postscript = "art"

print(f'Sample image by {evalprompt}\n')
#print(images)

raw_prompts = loadprompts((f"{root_path}/prompts.txt"))
prompts = []
post_prompts = []
for prompt in raw_prompts:
    prompts.append(prescript + " " + prompt)

for prompt in raw_prompts:
    post_prompts.append(prompt + " " + postscript)

if (evalprompt) not in raw_prompts:
    print('This is a new artist. Please let Lowfuel know so they can be added to the comparison list!')
    prompts.append(prescript + " " + evalprompt)
    post_prompts.append(evalprompt + " " + postscript)

for image in images:
    display.display(display.Image(fn))

average = 0.0
this_artist_score = []
for modelname in modellist:
    print("")
    print(f'{modelname} thinks this sample is:')
    scores = scoreprompts((prescript + " " + evalprompt), images, prompts, modelname)
    scores.sort(key=lambda a: a[1], reverse=True)
    for score in scores:
        name, percent = score
        if name == (prescript + " " + evalprompt):
            this_artist_score.append(percent)
        print(f'   {name} (with {percent:.2%} certainty)')

average = round(mean(this_artist_score),2)
print('')
print(f'Average CLIP recognition for {evalprompt} is {average:.2%}')

average = 0.0
this_artist_score = []
for modelname in modellist:
    print("")
    print(f'{modelname} thinks this sample is:')
    scores = scoreprompts((evalprompt + " " + postscript), images, post_prompts, modelname)
    scores.sort(key=lambda a: a[1], reverse=True)
    for score in scores:
        name, percent = score
        if name == (evalprompt + " " + postscript):
            this_artist_score.append(percent)
        print(f'   {name} (with {percent:.2%} certainty)')

average = round(mean(this_artist_score),2)
print('')
print(f'Average CLIP recognition for {evalprompt} is {average:.2%}')

"""
TODO:
Provide sample by URL (and allow for up to 4?)
Store artists average % recognized in the prompts file, read optionally
After image comparison, run cos similiarity against the other artists for both image and prompt
Provide final report (confidence, avg confidence, 4 similiar artists by image, 4 similar by text prompt, rec. clip models)
Look into better formatting of output
Use the batch iterable to ensure we don't hit any memory limit as the artist list grows
"""
